# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [2]:
import numpy as np
import pandas as pd
import math
import requests
import xlsxwriter
from scipy import stats

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [6]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [8]:
symbol ='AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'BLQ NT(EO SKLRACTAEASD NSMGA)GL/E',
 'calculationPrice': 'close',
 'open': 128.08,
 'openTime': 1654191264911,
 'openSource': 'aoiicflf',
 'close': 128.44,
 'closeTime': 1675824538397,
 'closeSource': 'iiflofca',
 'high': 123.41,
 'highTime': 1634722990778,
 'highSource': '1yer5neacl uem dip edti',
 'low': 122.57,
 'lowTime': 1638431250685,
 'lowSource': 'e1 rt5aeplmcie yu ieddn',
 'latestPrice': 127.14,
 'latestSource': 'Close',
 'latestTime': 'December 11, 2020',
 'latestUpdate': 1669466870662,
 'latestVolume': 91058622,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexLastUpdated': None,
 'delayedPrice': 127.77,
 'delayedPriceTime': 1627867067986,
 'oddLotDelayedPrice': 125.663,
 'oddLotDelayedPriceTime': 1634081998439,
 'extendedPrice': 126.1,
 'extendedChange': -0.14,
 'extendedChangePercent': -0.00117,
 'extendedPriceTime': 1630299311219,
 'previousClose': 126.43,
 'previousVolume': 85230478,
 'change':

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [9]:
price = data['latestPrice']
pe_ratio = data['peRatio']
pe_ratio

37.99

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [10]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [25]:
final_dataframe = pd.DataFrame(columns= my_columns)

for symbol_string in symbol_strings[:1]:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append (
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['peRatio'],
                    'N/A'
                ],
                index = my_columns
            ),
            ignore_index=True
        )
#data
final_dataframe 

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,123.64,54,N/A
1,AAL,17.25,-1.22,N/A
2,AAP,161.74,23.12,N/A
3,AAPL,125.08,38.56,N/A
4,ABBV,106.59,24.02,N/A
...,...,...,...,...
95,CINF,83.17,16.75,N/A
96,CL,86.38,27.83,N/A
97,CLX,207.24,23.23,N/A
98,CMA,53.25,14.62,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [26]:
final_dataframe.sort_values('Price-to-Earnings Ratio' ,inplace=True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe.drop('index', axis=1, inplace=True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BIO,569.86,4.9,N/A
1,ALL,105.90,7.65,N/A
2,BIIB,252.75,8.11,N/A
3,BK,40.52,9.15,N/A
4,AFL,46.55,11.34,N/A
5,C,59.67,11.61,N/A
6,BSX,34.60,11.91,N/A
7,ABC,101.82,13.38,N/A
8,AMP,196.69,13.93,N/A
9,BAC,28.58,14.46,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [27]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [28]:
portfolio_input()

Enter the value of your portfolio:1000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [29]:
position_size = float(portfolio_size)/len(final_dataframe.index)

for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row, 'Price'])
   
final_dataframe

    

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BIO,569.86,4.9,35
1,ALL,105.90,7.65,188
2,BIIB,252.75,8.11,79
3,BK,40.52,9.15,493
4,AFL,46.55,11.34,429
5,C,59.67,11.61,335
6,BSX,34.60,11.91,578
7,ABC,101.82,13.38,196
8,AMP,196.69,13.93,101
9,BAC,28.58,14.46,699


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [43]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

#Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']

#Price-to-book ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']
#pb_ratio

#Price-to-sales ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']
#ps_ratio

#Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
ev_to_ebitda = data[symbol]['advanced-stats']['enterpriseValue']/data[symbol]['advanced-stats']['EBITDA']
#ev_to_ebitda

#Enterprise Value divided by Gross Profit (EV/GP)
ev_to_gross_profit = data[symbol]['advanced-stats']['enterpriseValue']/data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit

19.29022291655538

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [95]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']

        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN

        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN

        rv_dataframe = rv_dataframe.append (
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    'N/A',
                    data[symbol]['quote']['peRatio'],
                    'N/A',
                    data[symbol]['advanced-stats']['priceToBook'],
                    'N/A',
                    data[symbol]['advanced-stats']['priceToSales'],
                    'N/A',
                    ev_to_ebitda,
                    'N/A',
                    ev_to_gross_profit,
                    'N/A',
                    'N/A',
                ],
                index = rv_columns
            ),
            ignore_index=True
        )


In [96]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,123.01,N/A,54.72,N/A,7.43,N/A,7.1,N/A,33.520953,N/A,13.161591,N/A,N/A
1,AAL,17.36,N/A,-1.24,N/A,-1.9,N/A,0.4327,N/A,-6.076090,N/A,1.424593,N/A,N/A
2,AAP,159.03,N/A,23.59,N/A,2.9,N/A,1.1,N/A,11.083216,N/A,2.525815,N/A,N/A
3,AAPL,125.73,N/A,37.71,N/A,31.9,N/A,7.67,N/A,26.852568,N/A,20.092375,N/A,N/A
4,ABBV,109.52,N/A,23.47,N/A,12.7,N/A,4.62,N/A,15.904078,N/A,9.611654,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.42,N/A,31.8,N/A,-4.06,N/A,5.97,N/A,23.512756,N/A,10.328475,N/A,N/A
501,ZBH,145.24,N/A,-192.8,N/A,2.59,N/A,4.29,N/A,24.691245,N/A,7.310188,N/A,N/A
502,ZBRA,396.08,N/A,45.13,N/A,10.5,N/A,4.84,N/A,30.618942,N/A,11.273778,N/A,N/A
503,ZION,42.13,N/A,18.37,N/A,0.9781,N/A,2.28,N/A,10.125338,N/A,2.595321,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [77]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
40,AON,208.900,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,235.430,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,9.88,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,26.140,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,51.230,N/A,14.66,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
168,EVRG,56.270,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,30.020,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRC,129.740,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
194,FTI,9.770,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,1784.210,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [97]:
for column in ['Price', 'Price-to-Earnings Ratio','Price-to-Book Ratio','Price-to-Sales Ratio','EV/EBITDA', 'EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace=True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [98]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,123.01,N/A,54.72,N/A,7.4300,N/A,7.1000,N/A,33.520953,N/A,13.161591,N/A,N/A
1,AAL,17.36,N/A,-1.24,N/A,-1.9000,N/A,0.4327,N/A,-6.076090,N/A,1.424593,N/A,N/A
2,AAP,159.03,N/A,23.59,N/A,2.9000,N/A,1.1000,N/A,11.083216,N/A,2.525815,N/A,N/A
3,AAPL,125.73,N/A,37.71,N/A,31.9000,N/A,7.6700,N/A,26.852568,N/A,20.092375,N/A,N/A
4,ABBV,109.52,N/A,23.47,N/A,12.7000,N/A,4.6200,N/A,15.904078,N/A,9.611654,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.42,N/A,31.80,N/A,-4.0600,N/A,5.9700,N/A,23.512756,N/A,10.328475,N/A,N/A
501,ZBH,145.24,N/A,-192.80,N/A,2.5900,N/A,4.2900,N/A,24.691245,N/A,7.310188,N/A,N/A
502,ZBRA,396.08,N/A,45.13,N/A,10.5000,N/A,4.8400,N/A,30.618942,N/A,11.273778,N/A,N/A
503,ZION,42.13,N/A,18.37,N/A,0.9781,N/A,2.2800,N/A,10.125338,N/A,2.595321,N/A,N/A


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [99]:
metrics = {
    'Price-to-Earnings Ratio' : 'PE Percentile' ,
    'Price-to-Book Ratio' : 'PB Percentile',
    'Price-to-Sales Ratio' : 'PS Percentile',
    'EV/EBITDA' : 'EV/EBITDA Percentile',
    'EV/GP': 'EV/GP Percentile'
}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

rv_dataframe        

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,123.01,N/A,54.72,0.861386,7.4300,0.726733,7.1000,0.815842,33.520953,0.855446,13.161591,0.813861,N/A
1,AAL,17.36,N/A,-1.24,0.141584,-1.9000,0.0455446,0.4327,0.0316832,-6.076090,0.0277228,1.424593,0.0554455,N/A
2,AAP,159.03,N/A,23.59,0.453465,2.9000,0.409901,1.1000,0.171287,11.083216,0.279208,2.525815,0.130693,N/A
3,AAPL,125.73,N/A,37.71,0.711881,31.9000,0.960396,7.6700,0.841584,26.852568,0.792079,20.092375,0.944554,N/A
4,ABBV,109.52,N/A,23.47,0.449505,12.7000,0.862376,4.6200,0.673267,15.904078,0.49703,9.611654,0.661386,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.42,N/A,31.80,0.639604,-4.0600,0.0415842,5.9700,0.766337,23.512756,0.738614,10.328475,0.689109,N/A
501,ZBH,145.24,N/A,-192.80,0.0217822,2.5900,0.368317,4.2900,0.617822,24.691245,0.766337,7.310188,0.469307,N/A
502,ZBRA,396.08,N/A,45.13,0.782178,10.5000,0.829703,4.8400,0.69802,30.618942,0.837624,11.273778,0.732673,N/A
503,ZION,42.13,N/A,18.37,0.360396,0.9781,0.0970297,2.2800,0.388119,10.125338,0.219802,2.595321,0.132673,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [100]:
from statistics import mean 

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
        
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,123.01,N/A,54.72,0.861386,7.4300,0.726733,7.1000,0.815842,33.520953,0.855446,13.161591,0.813861,0.814653
1,AAL,17.36,N/A,-1.24,0.141584,-1.9000,0.0455446,0.4327,0.0316832,-6.076090,0.0277228,1.424593,0.0554455,0.060396
2,AAP,159.03,N/A,23.59,0.453465,2.9000,0.409901,1.1000,0.171287,11.083216,0.279208,2.525815,0.130693,0.288911
3,AAPL,125.73,N/A,37.71,0.711881,31.9000,0.960396,7.6700,0.841584,26.852568,0.792079,20.092375,0.944554,0.850099
4,ABBV,109.52,N/A,23.47,0.449505,12.7000,0.862376,4.6200,0.673267,15.904078,0.49703,9.611654,0.661386,0.628713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.42,N/A,31.80,0.639604,-4.0600,0.0415842,5.9700,0.766337,23.512756,0.738614,10.328475,0.689109,0.57505
501,ZBH,145.24,N/A,-192.80,0.0217822,2.5900,0.368317,4.2900,0.617822,24.691245,0.766337,7.310188,0.469307,0.448713
502,ZBRA,396.08,N/A,45.13,0.782178,10.5000,0.829703,4.8400,0.69802,30.618942,0.837624,11.273778,0.732673,0.77604
503,ZION,42.13,N/A,18.37,0.360396,0.9781,0.0970297,2.2800,0.388119,10.125338,0.219802,2.595321,0.132673,0.239604


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [101]:
rv_dataframe.sort_values('RV Score', ascending=True, inplace=True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop=True, inplace=True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,23.27,N/A,5.21,0.156436,0.4480,0.0475248,0.4079,0.029703,2.588620,0.0336634,0.394129,0.0039604,0.0542574
1,AIG,39.28,N/A,-6.88,0.106931,0.5492,0.0554455,0.7522,0.0712871,4.284514,0.039604,0.694617,0.0158416,0.0578218
2,AAL,17.36,N/A,-1.24,0.141584,-1.9000,0.0455446,0.4327,0.0316832,-6.076090,0.0277228,1.424593,0.0554455,0.060396
3,MET,49.51,N/A,7.98,0.164356,0.5872,0.0574257,0.6755,0.0594059,4.403785,0.0415842,0.655963,0.0118812,0.0669307
4,HFC,28.47,N/A,-29.23,0.0673267,0.8550,0.0831683,0.3240,0.0158416,6.502994,0.0693069,2.291541,0.120792,0.0712871
5,F,9.46,N/A,-17.62,0.0811881,1.1700,0.134653,0.2825,0.0118812,4.701216,0.0435644,2.108972,0.112871,0.0768317
6,HIG,46.53,N/A,10.03,0.186139,0.9840,0.0990099,0.8072,0.0811881,4.031365,0.0376238,0.791109,0.0217822,0.0851485
7,ALL,106.06,N/A,7.32,0.160396,1.3100,0.170297,0.7173,0.0653465,2.315048,0.0316832,0.690901,0.0138614,0.0883168
8,KSS,38.61,N/A,-57.81,0.049505,1.3000,0.165347,0.3567,0.0237624,9.306432,0.182178,1.372697,0.0534653,0.0948515
9,BA,233.68,N/A,-29.42,0.0653465,-11.0000,0.029703,2.1800,0.372277,-30.646726,0.0138614,-219.662641,0.0019802,0.0966337


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [102]:
portfolio_input()

Enter the value of your portfolio:2500000


In [103]:
portfolio_size = float(portfolio_size)/len(rv_dataframe.index)
#portfolio_size

for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(portfolio_size/rv_dataframe.loc[row, 'Price'])
    
rv_dataframe

/Users/sumitchhabra/miniconda3/envs/algo/lib/python3.9/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,23.27,2148,5.21,0.156436,0.4480,0.0475248,0.4079,0.029703,2.588620,0.0336634,0.394129,0.0039604,0.0542574
1,AIG,39.28,1272,-6.88,0.106931,0.5492,0.0554455,0.7522,0.0712871,4.284514,0.039604,0.694617,0.0158416,0.0578218
2,AAL,17.36,2880,-1.24,0.141584,-1.9000,0.0455446,0.4327,0.0316832,-6.076090,0.0277228,1.424593,0.0554455,0.060396
3,MET,49.51,1009,7.98,0.164356,0.5872,0.0574257,0.6755,0.0594059,4.403785,0.0415842,0.655963,0.0118812,0.0669307
4,HFC,28.47,1756,-29.23,0.0673267,0.8550,0.0831683,0.3240,0.0158416,6.502994,0.0693069,2.291541,0.120792,0.0712871
5,F,9.46,5285,-17.62,0.0811881,1.1700,0.134653,0.2825,0.0118812,4.701216,0.0435644,2.108972,0.112871,0.0768317
6,HIG,46.53,1074,10.03,0.186139,0.9840,0.0990099,0.8072,0.0811881,4.031365,0.0376238,0.791109,0.0217822,0.0851485
7,ALL,106.06,471,7.32,0.160396,1.3100,0.170297,0.7173,0.0653465,2.315048,0.0316832,0.690901,0.0138614,0.0883168
8,KSS,38.61,1295,-57.81,0.049505,1.3000,0.165347,0.3567,0.0237624,9.306432,0.182178,1.372697,0.0534653,0.0948515
9,BA,233.68,213,-29.42,0.0653465,-11.0000,0.029703,2.1800,0.372277,-30.646726,0.0138614,-219.662641,0.0019802,0.0966337


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [104]:
writer = pd.ExcelWriter('value strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index=False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [105]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [106]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [107]:
writer.save()